In [3]:
import dask
dask.__version__

'2021.05.0'

In [8]:
%load_ext autoreload
%autoreload 2

import numpy as np
import dask
import time
import shutil
import warnings
import xarray as xr

import matplotlib.pyplot as plt
from fastjmd95 import rho
import fastjmd95

assert fastjmd95.__version__ >= "0.2.1"
# import fastjmd95 #quick fix? https://github.com/dask/distributed/issues/3450#issuecomment-584739501
# Nope
from multiprocessing.pool import ThreadPool
from dask.diagnostics import ProgressBar
import matplotlib.pyplot as plt

from xgcm import Grid
from fastprogress.fastprogress import progress_bar

from xarrayutils.file_handling import (
    write,
    maybe_create_folder,
    file_exist_check,
    temp_write_split,
)
from xarrayutils.utils import remove_bottom_values


from aguadv_omz_busecke_2021.vertical_transformation import (
    xgcm_transform_wrapper as transform_wrapper,
)
from aguadv_omz_busecke_2021.datasets import woa13
from aguadv_omz_busecke_2021.utils import print_html
from aguadv_omz_busecke_2021.preprocessing import (
    mixed_layer_wrapper,
    save_and_reload_rechunker,
    preprocessing_wrapper,
    strip_encoding,
)
from aguadv_omz_busecke_2021.omz_tools import (
    volume_consistency_checks,
    plot_omz_results,
    omz_thickness,
)
from aguadv_omz_busecke_2021.cmip6_stash import (
    detrend,
    construct_cfdate,
    _get_calendar,
    cmip6_dataset_id,
    combined_preprocessing,
    load_single_datasets,
    nested_dataset_wrapper,
    match_and_detrend,
    combine_grid_labels,
    parse_metrics,
)

dask.config.set(pool=ThreadPool(32))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
def transform_wrapper_again(
    ds_in,
    intensive_vars=[
        "thetao",
        "o2",
        "so",
        "agessc",
    ],
):
    # alternate name: transform mean wrapper
    # sigma_bins
    sigma_bins = np.array([0, 24.5, 26.5, 27.65, 100])

    # define variables to be averaged (intensive quantities)
    intensive_vars = [
        "thetao",
        "o2",
        "so",
        "agessc",
    ]  # add 'uo', 'agessc' etc?

    intensive_vars = [v for v in intensive_vars if v in ds_in.data_vars]

    for iv in intensive_vars:
        dz = (xr.ones_like(ds_in[iv]) * ds_in.dz_t).where(~np.isnan(ds_in[iv]))
        ds_in[iv] = ds_in[iv] * dz

    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        ds_out = transform_wrapper(
            ds_in,
            extensive_vars=["omz_thickness"] + intensive_vars,
            target=sigma_bins,
        )

    # reconvert the same variables
    dz = ds_out.dz_t
    for iv in intensive_vars:
        ds_out[iv] = ds_out[iv] / dz
    return ds_out

In [10]:
foldername = "density_remapping_obs"
ofolder = maybe_create_folder(f"/tigress/GEOCLIM/LRGROUP/samjd/global_omz/{foldername}")

In [11]:
woa = woa13().drop('time')
woa['sigma_0'] = (fastjmd95.rho(woa.so, woa.thetao, 0) - 1000)
woa = woa.rename({'thkcello':'dz_t'})
woa = woa.chunk({'lev':-1,'x':180})
woa['omz_thickness'] = omz_thickness(woa, o2_bins=o2_bins)

woa_sigma = transform_wrapper_again(woa, intensive_vars=[va for va in woa.data_vars if va not in ['omz_thickness']])

# confirm that the volume is conserved
with ProgressBar():
    vol_sigma = (woa_sigma.omz_thickness * woa_sigma.areacello).sum(['x','y','sigma_0']).load()
    vol_z = (woa.omz_thickness * woa.areacello).sum(['x', 'y', 'lev']).load()
    
assert (((vol_z - vol_sigma)/vol_z*100)<0.5).all()

fname = ofolder.joinpath('woa13.nc')
woa_sigma = write(woa_sigma, fname, overwrite=True)

/home/samjd/anaconda3/envs/cmip6_omz/lib/python3.9/site-packages/xarray/core/indexing.py:1379: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]
/home/samjd/anaconda3/envs/cmip6_omz/lib/python3.9/site-packages/xarray/core/indexing.py:1379: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[k

NameError: name 'warnings' is not defined